# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if we would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if we would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in our csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Design Tables to Query using Apache Cassandra.

#### Now we work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory. The <font color=red>event_datafile_new.csv</font> contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance our local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

## Create tables based on queries

#### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In this case, we use `sessionId` as the partition key. To make the primary key unique, we use `itemInSession` as a clustering column. The primary key is then (`sessionId`,`itemInSession`).

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS songplay_inSession "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
# Set up the CSV file. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT statements
        query = "INSERT INTO songplay_inSession (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## SELECT statement to verify the data was entered into the table
query = "select artist, song, length from songplay_inSession WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(columns=['artist','song_title','song_length'])

for row in rows:
    row_df = pd.DataFrame(data={'artist':[row.artist], 'song_title':[row.song], 'song_length':[row.length]})
    df = pd.concat([df, row_df], ignore_index=True)

df

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


In [11]:
query = "drop table songplay_inSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In this case, we use `sessionId` and `userId` as the partition key. To sort the song according to the request, i.e., by `itemInSession`, we use `itemInSession` as the clustering column. The primary key is then ((`sessionId`, `userId`), `itemInSession`). In addition, we specifically sort the results in a descending order of `itemInSession` by adding the phase `WITH CLUSTERING ORDER BY (itemInSession DESC)` at the end of the table definition.

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS user_song_inSession "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession DESC)"
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ##  INSERT statements 
        query = "INSERT INTO user_song_inSession (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]) ,int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                    

In [13]:
## SELECT statement to verify the data was entered into the table
query = "select artist, song, firstName, lastName from user_song_inSession WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(columns=['artist','song_title','user_firstName','user_lastName'])
    
for row in rows:
    row_df = pd.DataFrame(data={'artist':[row.artist], 'song_title':[row.song], 'user_firstName':[row.firstname], 'user_lastName':[row.lastname]})
    df = pd.concat([df, row_df], ignore_index=True)
df

,artist,song_title,user_firstName,user_lastName
0,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz
1,Sebastien Tellier,Kilometer,Sylvie,Cruz
2,Three Drives,Greece 2000,Sylvie,Cruz
3,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz


In [14]:
query = "drop table user_song_inSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In this case, we use `song` as the partition key. To make the primary key unique, we use `userId` as a clustering column since `userId` is directly related to users' first and last names. The primary key is then (`song`,`userId`).

In [15]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS user_song "
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT statements
        query = "INSERT INTO user_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))            

In [16]:
## SELECT statement to verify the data was entered into the table
query = "select firstName, lastName from user_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(columns=['user_firstName','user_lastName'])
    
for row in rows:
    row_df = pd.DataFrame(data={'user_firstName':[row.firstname], 'user_lastName':[row.lastname]})
    df = pd.concat([df, row_df], ignore_index=True)
df

,user_firstName,user_lastName
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


In [17]:
query = "drop table user_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close connection to the keyspace

In [18]:
session.shutdown()
cluster.shutdown()